In [68]:
import pandas as pd
import tensorflow as tf
import glob
import os
import numpy as np
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [69]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

# MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [70]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [71]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [72]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=True)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  # model.add(Dropout(drop))
  model.add(Dense(1, activation='linear')) #softmax

  #model.add(Lambda(lambda x: tf.cast(tf.argmax(x), tf.float32)))
  #model.add(Dense(1, activation=activ))

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32)                6272      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 8s 4ms/step - loss: 0.0987 - accuracy: 0.9537 - val_loss: 0.0309 - val_accuracy: 0.9893
Epoch 2/10
1536/1536 [==============================] - 5s 4ms/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.0171 - val_accuracy: 0.9927
Epoch 3/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.0095 - accuracy: 0.9968 - val_loss: 0.0023 - val_accuracy: 0.9996
Epoch 4/10
1536/

In [73]:
xTrain

array([[[ 1.        ,  0.41558932,  0.47934314, ...,  0.21069307,
         -0.29939312,  1.        ],
        [ 1.        ,  0.41913326,  0.4785721 , ...,  0.20987053,
         -0.30478759,  1.        ],
        [ 1.        ,  0.42063393,  0.47836566, ...,  0.2102361 ,
         -0.30478759,  1.        ],
        ...,
        [ 1.        ,  0.42518568,  0.46575717, ...,  0.20715918,
         -0.32501686,  1.        ],
        [ 1.        ,  0.42823233,  0.45011742, ...,  0.20706778,
         -0.32164531,  1.        ],
        [ 1.        ,  0.43632028,  0.43766773, ...,  0.20743336,
         -0.32501686,  1.        ]],

       [[ 1.        ,  0.42619791,  0.47609839, ...,  0.2243412 ,
         -0.2744437 ,  0.08333333],
        [ 1.        ,  0.43999461,  0.46098267, ...,  0.22470678,
         -0.28523264,  0.08333333],
        [ 1.        ,  0.44367336,  0.46035101, ...,  0.22528561,
         -0.28792987,  0.08333333],
        ...,
        [ 1.        ,  0.42564649,  0.4676645 , ...,  

In [74]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Lie/processed/'

test_lstm.perdictSingleVideo(lie_test_path, MODEL)

[[    gaze_0_x  gaze_0_y  gaze_0_z  gaze_angle_x  gaze_angle_y  pose_Tx  \
35  0.165284  0.032253 -0.985718         0.086         0.160    -58.3   
36  0.168286  0.008629 -0.985700         0.110         0.118    -59.0   
37  0.155668  0.003196 -0.987804         0.108         0.093    -59.9   
38  0.135473  0.022942 -0.990515         0.086         0.097    -60.8   
39  0.156147  0.016189 -0.987601         0.093         0.094    -61.9   
40  0.156607  0.021641 -0.987424         0.093         0.098    -62.8   
41  0.173506  0.007787 -0.984802         0.103         0.090    -63.7   
42  0.157068  0.036344 -0.986919         0.103         0.108    -64.5   
43  0.165952  0.038948 -0.985364         0.095         0.121    -65.6   
44  0.152045  0.045429 -0.987329         0.089         0.114    -66.6   

    pose_Ty  pose_Tz  pose_Ry  AU01_r  AU04_r  AU10_r  AU12_r  AU45_r  \
35     55.8    586.6    0.200    0.05    0.49    0.24    0.26    1.17   
36     56.4    585.2    0.195    0.00    0.51   

Traceback (most recent call last):
  File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_78230/2887895029.py", line 6, in <cell line: 6>
    test_lstm.perdictSingleVideo(lie_test_path, MODEL)
  File "/Users/frank/Downloads/Projects/Lie Detection/Lie_Detection/test_lstm.py", line 41, in perdictSingleVideo
    modelObj.predict(data)
  File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/__autograph_generated_file5e_jvs6i.py", line 15, in tf__predict_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/Users/frank/opt/anaconda3/lib/p